# Nystroem Kernel Approximation


<table class="nt-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://MatchCake.github.io/MatchCake/"><img src="https://github.com/MatchCake/MatchCake/blob/main/images/logo/logo_32px.png?raw=true" width=32px height=32px  />Documentation</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/MatchCake/MatchCake/blob/main/tutorials/iris_classification.ipynb"><img src="https://github.com/MatchCake/MatchCake/blob/main/images/logo/colab_logo_32px.png?raw=true" width=32px height=32px  />Run in Google Colab</a>
</td>
  <td>
    <a target="_blank" href="https://github.com/MatchCake/MatchCake/blob/main/tutorials/iris_classification.ipynb"><img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=32px height=32px />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/MatchCake/MatchCake/blob/main/tutorials/iris_classification.ipynb"><img src="https://github.com/MatchCake/MatchCake/blob/main/images/logo/download_logo_32px.png?raw=true" width=32px height=32px />Download notebook</a>
  </td>
</table>

# Setup

You can now install the dependencies by running the following commands:

In [ ]:
#@title Install dependencies {display-mode: "form"}

RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

if RunningInCOLAB:
    !pip install matchcake

# Imports

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from matchcake.ml.kernels import FermionicPQCKernel
from matchcake.ml.visualisation import ClassificationVisualizer
from matchcake.ml.kernels.nystroem import Nystroem

# Parameters

We will define our hyperparameters here. We will use 4 qubits to encode the data.

In [ ]:
n_qubits = 12

# Load the data

In [ ]:
dataset = datasets.load_digits(as_frame=True)
X, y = dataset.data, dataset.target
print(f"Data shape: X: {X.shape}, y: {y.shape}")

# Split the data

We will split the data into training and testing sets.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Build the SKlearn Pipeline

We will build the model using the `FermionicPQCKernel`.

In [ ]:
pipline = Pipeline([
    ('scaler', MinMaxScaler(feature_range=(0, 1))),
    (
        'nystroem_kernel',
        Nystroem(
            FermionicPQCKernel(n_qubits=n_qubits, rotations="X,Z").freeze(),
            n_components=50,
            random_state=0,
        ).freeze()
     ),
    ('classifier', LinearSVC()),
])

# Train the model

We will train the model using the training data.

In [ ]:
pipline.fit(x_train, y_train)

# Evaluate the Pipeline

We will evaluate the pipeline using the testing data.

In [ ]:
test_accuracy = pipline.score(x_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# Visualize the results

We can visualize the decision boundaries of the model.

In [ ]:
viz = ClassificationVisualizer(x=X, n_pts=1_000)
fig, ax, y_pred = viz.plot_2d_decision_boundaries(
    model=pipline, y=y, legend_labels=getattr(dataset, "target_names", None), show=True
)

----------